In [2]:
from datascience import *
import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import plotly.express as px
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

In [3]:
stats = Table.read_table("538 Ratings.csv")
stats

YEAR,TEAM NO,TEAM,SEED,ROUND,POWER RATING,POWER RATING RANK
2023,993,Houston,1,16,93.2,13
2023,1011,Alabama,1,16,92.6,16
2023,986,Kansas,1,32,89.6,49
2023,963,Purdue,1,64,89.5,50
2023,957,Texas,2,8,90.1,43
2023,1010,Arizona,2,64,89,59
2023,952,UCLA,2,16,88.3,78
2023,980,Marquette,2,32,87.6,92
2023,995,Gonzaga,3,8,89.9,46
2023,1006,Baylor,3,32,87.1,104


In [4]:
stats_cleaned1 = stats.select('SEED', 'ROUND', 'POWER RATING', 'POWER RATING RANK')
stats_cleaned1

SEED,ROUND,POWER RATING,POWER RATING RANK
1,16,93.2,13
1,16,92.6,16
1,32,89.6,49
1,64,89.5,50
2,8,90.1,43
2,64,89,59
2,16,88.3,78
2,32,87.6,92
3,8,89.9,46
3,32,87.1,104


In [5]:
stats = stats.with_columns('SEED_SU', standard_units(stats_cleaned1.column('SEED')),
                                   'ROUND_SU', standard_units(stats_cleaned1.column('ROUND')),
                                   'POWER_RATING_SU', standard_units(stats_cleaned1.column('POWER RATING')),
                                    'POWER_RATING_RANK_SU', standard_units(stats_cleaned1.column('POWER RATING RANK'))).drop('SEED', 'ROUND', 'POWER RATING', 'POWER RATING RANK')
stats

YEAR,TEAM NO,TEAM,SEED_SU,ROUND_SU,POWER_RATING_SU,POWER_RATING_RANK_SU
2023,993,Houston,-1.66696,-1.22951,1.73284,-1.63181
2023,1011,Alabama,-1.66696,-1.22951,1.63836,-1.60999
2023,986,Kansas,-1.66696,-0.530967,1.16598,-1.37002
2023,963,Purdue,-1.66696,0.866112,1.15023,-1.36275
2023,957,Texas,-1.45297,-1.57878,1.24471,-1.41365
2023,1010,Arizona,-1.45297,0.866112,1.0715,-1.2973
2023,952,UCLA,-1.45297,-1.22951,0.961277,-1.15914
2023,980,Marquette,-1.45297,-0.530967,0.851054,-1.05733
2023,995,Gonzaga,-1.23898,-1.57878,1.21322,-1.39184
2023,1006,Baylor,-1.23898,-0.530967,0.772323,-0.970067


In [6]:
def row_distance(row1, row2):
    """Return the distance between two numerical rows of a table."""
    return distance(np.array(row1), np.array(row2))

def distance(pt1, pt2):
    """Return the distance between two points, represented as arrays"""
    return np.sqrt(sum((pt2-pt1)**2))

ideal_row = Table().with_columns('SEED_SU', min(stats.column('SEED_SU')), 'ROUND', min(stats.column('ROUND_SU')), 'POWER_RATING_SU', max(stats.column('POWER_RATING_SU')),'POWER_RATING_RANK_SU', min(stats.column('POWER_RATING_RANK_SU')))

In [7]:
def matchup(data, team1, team2):
    """Return team who won."""
    team1_row = data.where('TEAM', are.equal_to(team1)).where('YEAR', are.equal_to(2022))
    team1_row = team1_row.drop('YEAR', 'TEAM NO', 'TEAM')
    team2_row = data.where('TEAM', are.equal_to(team2)).where('YEAR', are.equal_to(2017))
    team2_row = team2_row.drop('YEAR', 'TEAM NO', 'TEAM')
    team1_dist = row_distance(team1_row.row(0), ideal_row.row(0))
    team2_dist = row_distance(team2_row.row(0), ideal_row.row(0))
    if team1_dist < team2_dist:
        return team1 + '1'
    else:
        return team2
                              

In [10]:
matchup(stats, 'Duke', 'North Carolina')


'North Carolina'

In [37]:
stats_2021_2024 = stats.where('YEAR', are.above(2021))

distances = make_array()
stats_reduced = stats_2021_2024.drop('YEAR', 'TEAM NO', 'TEAM')
    
for i in np.arange(stats_2021_2024.num_rows):
    row = stats_reduced.row(i)
    distances = np.append(distances, row_distance(row, ideal_row.row(0)))
        
stats_2021_2024.with_column('Distance_to_ex', distances).sort('Distance_to_ex').show(136)



YEAR,TEAM NO,TEAM,SEED_SU,ROUND_SU,POWER_RATING_SU,POWER_RATING_RANK_SU,Distance_to_ex
2022,925,Gonzaga,-1.66696,-1.22951,2.25246,-1.71907,0.654881
2022,918,Kansas,-1.66696,-1.88439,1.43366,-1.53728,0.83874
2023,993,Houston,-1.66696,-1.22951,1.73284,-1.63181,0.840531
2023,1011,Alabama,-1.66696,-1.22951,1.63836,-1.60999,0.904371
2022,881,Villanova,-1.45297,-1.75341,1.26045,-1.42093,1.0658
2023,957,Texas,-1.45297,-1.57878,1.24471,-1.41365,1.11716
2022,941,Arizona,-1.66696,-1.22951,1.37068,-1.4791,1.12428
2023,995,Gonzaga,-1.23898,-1.57878,1.21322,-1.39184,1.20983
2022,927,Duke,-1.45297,-1.75341,1.11874,-1.33366,1.22345
2022,924,Houston,-1.66696,-1.57878,1.02426,-1.22458,1.35882
